<a href="https://colab.research.google.com/github/Pataweepr/applyML_vistec_2019/blob/master/hw4_virus_classification_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Virus Classification using Support Vector Machines (SVM)

In this lab we will classify viruses into their sub families using their DNA sequence. This has potential applications in disease control where scientists can monitor the spread and mutations of viruses, and potentially try to identify whether when a benign virus will mutate to a malignant virus.

For this lab we will mainly use Support Vector Machines ([SVM](https://scikit-learn.org/stable/modules/svm.html))

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import collections
import csv

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import seaborn as sns
from IPython.display import display
from scipy.stats import mode

from sklearn.svm import SVC

seed = 976
np.random.seed(seed)

import time

## The data

The data can be found [here](https://drive.google.com/file/d/1tb1pvtUNqx3r4FVbNsRKum2hwaLOIYmO/view?usp=sharing). Click add to drive so that you can mount it later.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/Chosen_Data_clearToN.csv")

### Data preparation

For the data provided, we will only use the "Sequences" and the "Label" columns.

**Sequences** refers to the DNA sequence of a virus sample.

**Label** refers to the classification of the virus.

Create a dataframe that only have the two columns.

How many kinds of viruses are we working with?

**Ans:**

All these viruses are different variations of Ebola viruses.

In [0]:
## TODO#1 ##


Each DNA string has different length. Plot the histogram of the length of the DNA.

In [0]:
## TODO#2 ##


## Handling input of different sizes (text)

In machine learning, it is oftened assume that the input features are of the same length for each training data (since the model is a function which requires fixed-length inputs). However, in this case we have variable-length inputs. A naive solution would be to pad (add blank characters) so that the input will be the same length. This can sometimes create unwanted effects. For example in this case, most sequence will be padded to 38000 which causes the sequence after the 32000th position to be mostly blank characters for most data sample. Another solution would be to use models that can handle variable-length inputs, for example recurrent neural networks. In this homework we will use another powerful concept called, bag-of-words (BoW) representation.

### BoW

BoW refers to the representation of character counts a string.

For example, the string

"apple"

can be represented as counts below:

Char | Counts
--- | ---
a | 1
p | 2
l | 1
e | 1

This representation is a bit too simple. People can add richness to the representation by counting every subsequence of length n as shown below:

substring(n=2) | Counts
--- | ---
ap | 1
pp | 1
pl | 1
le | 1

This is called [n-gram](https://en.wikipedia.org/wiki/N-gram) representation (n=2). For the field of bioinformatics, this representation is usually refered to as [k-mer](https://en.wikipedia.org/wiki/K-mer) (k=2).


Train, validation, test split can be done using the function below. Note how the split is done one class at a time. This is done so that the portion of classes in each subset are the same. This is called stratified splitting.

Sk-learn also have functions that accomplish this. You can look at it [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html).

In [0]:
def splitTrainTest(data):
  keyDatas = data["Label"].value_counts().keys()
  train = pd.DataFrame()
  valid = pd.DataFrame()
  test = pd.DataFrame()
  chk = 0
  for k in keyDatas:
    tmp = data[data["Label"]==k]
    tmp_train, tmp_test = train_test_split(tmp, test_size=2/5, random_state=seed)
    tmp_train, tmp_valid = train_test_split(tmp_train, test_size=1/6, random_state=seed)
    train = train.append(tmp_train)
    valid = valid.append(tmp_valid)
    test = test.append(tmp_test)
  return train, valid, test

In [0]:
df_train, df_valid, df_test = splitTrainTest(df)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

Why should we do stratified splitting? (hint: think about how we do hyperparameter tuning).

** Ans:**

Check if the data in each subset has the correct propotion for each class.

What is the majority class? What is the minority class? What is the ratio in terms of number of training data between majority and minority classes?

** Ans: **

In [0]:
## TODO#3 ##


## N-gram generation

Write a function that takes in a DNA sequence and return an n-gram of ATCG combinations.

You can use [itertools.product](https://www.hackerrank.com/challenges/itertools-product/problem) to help create n-gram permutations.

Note: besides ATCG, there is also the letter "N." This is an unknown base. Ignore any n-gram that contains "N."

Hint: use a dictionary to keep track of counts for each n-gram.

In [0]:
def createGram(sequence, gram=5):
  # sequence is the DNA sequence
  # returns: np array where each entry is an n-gram count

  ## TODO#4 ##
  return ngramdata

In [0]:
# Try
print(createGram('AGTCATC',1))
print(createGram('AGTCATC',2))

#You should get
#[2 2 2 1]
#[0 0 1 1 1 0 0 0 0 2 0 0 0 0 1 0]

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
def createGram(sequence, gram=5):
  nGram = dict.fromkeys(["".join(x) for x in itertools.product("ACTG",repeat=gram) ],0)
  for i in  range(len(sequence)-gram+1):
    if "N" not in str([sequence[i:i+gram]]):
      nGram[sequence[i:i+gram]]+=1
  return np.array(list(nGram.values()))
        </code>
      </pre>
</details>

createGramDataset() is a function that calls createGram to generate n-gram features for the entire dataset. For each training data, we take the n-gram features from a  **random** sub-sequence of length t (default value = 10000). 

It can also upsampling the data to deal with our imbalanced dataset. To generate extra training examples for the class with smaller amounts of training data we can take a larger number of sub-sequences per training data.



In [0]:
def createGramDataset(data, gram=5, length=10000, train=False):
  datangram = []
  label = []

  if(train):
    # Upsampling if training data
    
    # Assume first class has maximum amount
    max_sam = data["Label"].value_counts()[0]
    # For each label
    for i in data["Label"].value_counts().keys():
      tmp_virus = data[data["Label"]==i]
      tmp_virus = tmp_virus.reset_index(drop=True)
      # Upsample by j times
      for j in range(0, max_sam//len(tmp_virus)):
        # For data point
        for k in range(len(tmp_virus)):
          # Select location of the sub-sequence
          if(len(tmp_virus["Sequences"][k])-length == 0):
            rand_int = 0
          else:
            rand_int = np.random.randint(len(tmp_virus["Sequences"][k])-length)
          selected_sequence = tmp_virus["Sequences"][k][rand_int:rand_int+length]
          datangram.append(createGram(tmp_virus["Sequences"][k]))
          label.append(i)
  else:
    # For data point
    for k in range(len(data)):
      if (len(data["Sequences"][k])-length == 0) :
        rand_int = 0
      else:
        rand_int = np.random.randint(len(data["Sequences"][k])-length)
      selected_sequence = data["Sequences"][k][rand_int:rand_int+length]
      datangram.append(createGram(data["Sequences"][k]))
      label.append(data["Label"][k])

  return np.array(datangram), label

In [0]:
X_train, y_train = createGramDataset(df_train, 5, 20000, True)
X_valid, y_valid = createGramDataset(df_valid, 5, 20000, False)
X_test, y_test = createGramDataset(df_test, 5, 20000, False)

buildlLabel() is a function that maps categorical values to numbers.

In [0]:
def buildLabel(y_str_label,key_virus_label):
  nplabel = []
  for lab in y_str_label:
    for i in np.arange(len(key_virus_label)):
      if(lab == key_virus_label[i]):
        nplabel.append(i+1)
        break
  return np.array(nplabel)

In [0]:
key_virus = df["Label"].value_counts().index
y_train = buildLabel(y_train,key_virus)
y_valid = buildLabel(y_valid,key_virus)
y_test = buildLabel(y_test,key_virus)
print(y_train)

In [0]:
# Shuffle the training data so that classes will not appear together.
X_train, y_train = shuffle(X_train, y_train, random_state=seed)

In [0]:
# A function that spits out several metrics
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def eval(y_pred,y_test):
  acc = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred, average='macro') 
  prec = precision_score(y_test, y_pred, average='macro') 
  recall = recall_score(y_test, y_pred, average='macro') 
  return acc, f1, prec, recall

## Linear Kernal (SVC)

Use [SVM](https://scikit-learn.org/stable/modules/svm.html) in sk-learn with linear kernel to train a model with default parameters. Report the accuracy, f1, precision, and recall using the function eval().

In [0]:
## TODO#5 ##


<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
clf_Linear = SVC(kernel='linear')
clf_Linear.fit(X_train, y_train)
y_pred_linear = clf_Linear.predict(X_test)

acc, f1, precision, recall = eval(y_pred_linear,y_test)
print('accuracy :',acc)
print('f1 :',f1)
print('precision :',precision)
print('recall :',recall)
        </code>
      </pre>
</details>

## Polynomial Kernel

Try polynomail with polynomial degree 2, 3, and 4. The degree of the polynomial is typically a hyperparameter to tune. You may find the [doc](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) helpful.

Report the difference on the validation and the test sets.

Which one should you use?

** Ans: **

In [0]:
## TODO#6 ##


## Radial Basis Kernel (RBF)

Finally, try RBF kernel.

The parameter gamma refers to the size of our Gaussian kernel. A higher Gamma means smaller Guassian, and vice versa. In other words, the larger gamma is, the closer other examples must be to be affected.

Try different gamma values and plot the accuracy of the training, validation, and the test set as a function of the parameter Gamma.

Use Gamma = $10^x$ where $x$ is from {-4, -3, ..., 3, 4}

In [0]:
## TODO#7 ##


After you finish the plot, do you think we need to further tune the hyperparameter gamma?

** Ans: **

What is the relation between overfitting and the hyperparameter gamma?

** Ans: **

Which kernel is the best for this dataset?

** Ans: **

*The type of kernel that should be used depends on the dataset. This is also a kind of hyperparameter. Also make sure you tune the hyperparameters for each kernel. *

## Finding the best 2000 

DNA sequencing can also be done on part of the sequence. This can significantly reduce the cost of DNA sequencing. Find the best sub-sequent of length 2000, so that we can reduce the cost of sequencing.

Suppose according to your biologist friend, there are 2 regions of significance:

1.   Position 6000-8000
2.   Position 8000-10000

Which region is the better one?

In [0]:
# Example code of how to get 8000:10001
df_cut = df.copy()
sub_Sequences = [x[8000:10001] for x in df_cut['Sequences'].values]
df_cut['sub_Sequences'] = sub_Sequences
df_cut = df_cut[['sub_Sequences','Label']]

df_cut.rename(columns={'sub_Sequences': 'Sequences'}, inplace=True)

In [0]:
## TODO#8 ##

Which region is the better one?

** Ans: **

## Slack penalty C

Another hyperparameter that should be tuned is the slack penalty. In practice this should be tuned according to each Kernel, but we have left them at the default values until now.

Slack values refer to regularization term that allows some examples to be misclassified. This reduces overfitting. A low C has high regularization and makes the decision surface smooth, while a high C aims at classifying all training examples correctly (low slack).

Another thing to note is the number of support vectors. The decision boundary of SVMs are built using support vectors. Kernel SVM relies on the calculation over all support vectors, so the more the support vectors the higher the runtime. This is also one of the reason why SVMs which do very well in small datasets do not see much practical usage. The bigger the dataset, the bigger amount of support vectors, resulting in higher runtime.

In this section, we will look at the relationship between the slack penalty, the number of support vectors, and the runtime.

Train a RBF SVM (with default parameters), use C = $10^x$ where $x$ is from {-4, -3, ..., 3, 4}.

Monitor the amount of support vectors (clf.n_support_) and the runtime for each value C.

You can monitor the runtime by

```
start_time = time.time()
# run 10 times to get the average
for runs in range(10):
    clf.fit(X_train_key_seq, y_train_key_seq)
end_time = time.time()
avg_time = (start_time - end_time)/10
```

In [0]:
## TODO#9 ##


<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
df_train, df_valid, df_test = splitTrainTest(df)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

X_train_key_seq, y_train_key_seq = createGramDataset(df_train, 5, 20000, True)
X_valid_key_seq, y_valid_key_seq = createGramDataset(df_valid, 5, 20000, False)
X_test_key_seq, y_test_key_seq = createGramDataset(df_test, 5, 20000, False)

y_train_key_seq = buildLabel(y_train_key_seq,key_virus)
y_valid_key_seq = buildLabel(y_valid_key_seq,key_virus)
y_test_key_seq = buildLabel(y_test_key_seq,key_virus)

X_train_key_seq, y_train_key_seq = shuffle(X_train_key_seq, y_train_key_seq, random_state=seed)

train_acc = [] 
test_acc = []
number_sv = []
runtime = []

a = range(-4,4)

for idx,val in enumerate(a):
  clf = SVC(gamma='auto',C=10**val)
  start_time = time.time()
  for runs in range(10):
    clf.fit(X_train_key_seq, y_train_key_seq)
  end_time = time.time()
  y_pred_train = clf.predict(X_train_key_seq)
  y_pred = clf.predict(X_test_key_seq)
  
  number_sv.append(np.sum(clf.n_support_))
  acc_train, _, _, _ = evluate(y_pred_train,y_train_key_seq)
  acc_test, _, _, _ = evluate(y_pred,y_test_key_seq)
  train_acc.append(acc_train)
  test_acc.append(acc_test)
  runtime.append((end_time - start_time)/10)
  
plt.plot(a, np.array(train_acc), 'r', a, np.array(test_acc), 'b')
plt.legend(('train', 'test'),loc='upper right')
plt.show()

plt.plot(a, np.array(number_sv), 'r')
plt.show()

plt.plot(a, np.array(runtime), 'r')
plt.show()
        </code>
      </pre>
</details>

According to the graphs what is the relationship between

- Overfitting and C
- C and number of support vectors
- number of support vectors and runtime

** Ans: **

## (Optional) New-virus identification using one-class SVM ([NuSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html))

It is often very important to be able to identify when a new type of virus emerges. This type of problem is called Anomaly detection in machine leanring literature. Anomaly detection can be framed as a one class problem, where we reject every data point that is too far from the group of known classes.

You can try leaving one classes of viruses out and create a classifier using the rest of the classes. Create a NuSVC model to identify whether the left out virus is a new type or not.

Things to think about:

- How to split the training, validation, and test data in this case?
- How unique is each class? Can we quantify it?
- Can we identify which pair of classes are closer than the rest?
- Given time stamps of the samples, can we map out how these viruses evolve from each family?